In [2]:
import requests
import time
import pandas as pd

API_KEY = "73af62f1-882f-4c88-a477-ce6f38adde01"

def make_api_request(url):
    headers = {"api_key": API_KEY}
    
    while True:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.HTTPError as e:
            if e.response.status_code == 429:
                retry_after = int(e.response.headers.get('Retry-After', 60))
                print(f"Rate limit reached. Waiting for {retry_after} seconds before retrying...")
                time.sleep(retry_after)
            else:
                print(f"HTTP error occurred: {e}")
                return None
        except requests.RequestException as e:
            print(f"An error occurred: {e}")
            return None

def get_supported_exchanges():
    url = "https://api.coinalyze.net/v1/exchanges"
    return make_api_request(url)

def create_exchanges_dataframe(exchanges):
    return pd.DataFrame(exchanges)

def get_supported_future_markets():
    url = "https://api.coinalyze.net/v1/future-markets"
    return make_api_request(url)

def create_supported_futures_markets_dataframe(future_markets):
    df = pd.DataFrame(future_markets)
    
    # Convert expire_at from milliseconds to datetime
    df['expire_at'] = pd.to_datetime(df['expire_at'], unit='ms')
    
    return df

def filter_btc_perpetual_futures(futures_df):
    # Filter the DataFrame for BTC base asset, perpetual futures, and USD-based quote assets
    btc_perpetual_df = futures_df[
        (futures_df['base_asset'] == 'BTC') & 
        (futures_df['is_perpetual'] == True) &
        (futures_df['quote_asset'].isin(['USD', 'USDT', 'USDC']))
    ].copy()  # Create a copy to avoid SettingWithCopyWarning
    
    return btc_perpetual_df

def add_exchange_names(btc_perpetual_df, exchanges_df):
    # Create a dictionary mapping exchange codes to names
    exchange_dict = dict(zip(exchanges_df['code'], exchanges_df['name']))
    
    # Add a new column 'exchange_name' by mapping the 'exchange' column
    btc_perpetual_df['exchange_name'] = btc_perpetual_df['exchange'].map(exchange_dict)
    
    return btc_perpetual_df

def main():
    # Fetch and process exchanges
    exchanges = get_supported_exchanges()
    if exchanges:
        exchanges_df = create_exchanges_dataframe(exchanges)
        #print("Supported Exchanges (first 5 rows):")
        #print(exchanges_df.head())
        exchanges_df.to_csv('supported_exchanges.csv', index=False)
        print("Exchanges DataFrame saved to 'supported_exchanges.csv'")
    else:
        print("Failed to retrieve supported exchanges.")
        return

    # Fetch and process future markets
    future_markets = get_supported_future_markets()
    if future_markets:
        futures_df = create_supported_futures_markets_dataframe(future_markets)
        #print("\nSupported Future Markets (first 5 rows):")
        #print(futures_df.head())
        futures_df.to_csv('supported_future_markets.csv', index=False)
        print("Future Markets DataFrame saved to 'supported_future_markets.csv'")
        
        # Filter for BTC perpetual futures with USD-based quote assets
        btc_perpetual_df = filter_btc_perpetual_futures(futures_df)
        
        # Add exchange names to the BTC perpetual futures DataFrame
        btc_perpetual_df = add_exchange_names(btc_perpetual_df, exchanges_df)
        
        #print("\nBTC Perpetual Futures with USD-based quote assets (first 5 rows):")
        #print(btc_perpetual_df.head())
        btc_perpetual_df.to_csv('btc_perpetual_futures_usd.csv', index=False)
        print("BTC Perpetual Futures DataFrame saved to 'btc_perpetual_futures_usd.csv'")
    else:
        print("Failed to retrieve supported future markets.")

if __name__ == "__main__":
    main()

Supported Exchanges (first 5 rows):
       name code
0  Poloniex    P
1    Vertex    V
2  Bitforex    D
3    Kraken    K
4   Bithumb    U
Exchanges DataFrame saved to 'supported_exchanges.csv'

Supported Future Markets (first 5 rows):
           symbol exchange symbol_on_exchange base_asset quote_asset  \
0       MNTUSDT.6        6            MNTUSDT        MNT        USDT   
1       QNTUSDT.6        6            QNTUSDT        QNT        USDT   
2       MAVUSDT.6        6            MAVUSDT        MAV        USDT   
3  NMRUSDT_PERP.A        A            NMRUSDT        NMR        USDT   
4        L3USDT.6        6             L3USDT         L3        USDT   

  expire_at  has_buy_sell_data  is_perpetual margined  \
0       NaT               True          True   STABLE   
1       NaT               True          True   STABLE   
2       NaT               True          True   STABLE   
3       NaT               True          True   STABLE   
4       NaT               True          True   